# Imports

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Logistic Regression

Let's return to the ${\tt Smarket}$ data. 

In [2]:
df = pd.read_csv('Smarket.csv', usecols=range(1,10), index_col=0, parse_dates=True)
df.describe()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.00000,1250.000000,1250.000000
mean,0.003834,0.003919,0.001716,0.001636,0.00561,1.478305,0.003138
std,1.136299,1.136280,1.138703,1.138774,1.14755,0.360357,1.136334
min,-4.922000,-4.922000,-4.922000,-4.922000,-4.92200,0.356070,-4.922000
25%,-0.639500,-0.639500,-0.640000,-0.640000,-0.64000,1.257400,-0.639500
50%,0.039000,0.039000,0.038500,0.038500,0.03850,1.422950,0.038500
75%,0.596750,0.596750,0.596750,0.596750,0.59700,1.641675,0.596750
max,5.733000,5.733000,5.733000,5.733000,5.73300,3.152470,5.733000


In [3]:
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In this lab, we will fit a logistic regression model in order to predict ${\tt Direction}$ using ${\tt Lag1}$ through ${\tt Lag5}$ and ${\tt Volume}$. We'll build our model using the ${\tt glm()}$ function, which is part of the
${\tt formula}$ submodule of (${\tt statsmodels}$).

In [4]:
import statsmodels.formula.api as smf

We can use an ${\tt R}$-like formula string to separate the predictors from the response.

In [5]:
formula = 'Direction ~ Lag1+Lag2+Lag3+Lag4+Lag5+Volume'

The ${\tt glm()}$ function fits **generalized linear models**, a class of models that includes logistic regression. The syntax of the ${\tt glm()}$ function is similar to that of ${\tt lm()}$, except that we must pass in the argument ${\tt family=sm.families.Binomial()}$ in order to tell ${\tt R}$ to run a logistic regression rather than some other type of generalized linear model.

In [6]:
model = smf.glm(formula=formula, data=df, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

                          Generalized Linear Model Regression Results                           
Dep. Variable:     ['Direction[Down]', 'Direction[Up]']   No. Observations:                 1250
Model:                                              GLM   Df Residuals:                     1243
Model Family:                                  Binomial   Df Model:                            6
Link Function:                                    logit   Scale:                             1.0
Method:                                            IRLS   Log-Likelihood:                -863.79
Date:                                  Tue, 11 Apr 2017   Deviance:                       1727.6
Time:                                          14:26:49   Pearson chi2:                 1.25e+03
No. Iterations:                                       4                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------

The smallest p-value here is associated with ${\tt Lag1}$. The negative coefficient
for this predictor suggests that if the market had a positive return yesterday,
then it is less likely to go up today. However, at a value of 0.145, the p-value
is still relatively large, and so there is no clear evidence of a real association
between ${\tt Lag1}$ and ${\tt Direction}$.

We use the ${\tt .params}$ attribute in order to access just the coefficients for this
fitted model. Similarly, we can use ${\tt .pvalues}$ to get the p-values for the coefficients, and ${\tt .model.endog_names}$ to get the **endogenous** (or dependent) variables.

In [7]:
print("Coeffieients")
print(result.params)
print
print("p-Values")
print(result.pvalues)
print
print("Dependent variables")
print(result.model.endog_names)

Coeffieients
Intercept    0.126000
Lag1         0.073074
Lag2         0.042301
Lag3        -0.011085
Lag4        -0.009359
Lag5        -0.010313
Volume      -0.135441
dtype: float64
p-Values
Intercept    0.600700
Lag1         0.145232
Lag2         0.398352
Lag3         0.824334
Lag4         0.851445
Lag5         0.834998
Volume       0.392404
dtype: float64
Dependent variables
['Direction[Down]', 'Direction[Up]']


Note that the dependent variable has been converted from nominal into two dummy variables: ${\tt ['Direction[Down]', 'Direction[Up]']}$.

The ${\tt predict()}$ function can be used to predict the probability that the
market will go down, given values of the predictors. If no data set is supplied to the
${\tt predict()}$ function, then the probabilities are computed for the training
data that was used to fit the logistic regression model. 

In [10]:
predictions = result.predict()
print(predictions[0:10])

[ 0.49291587  0.51853212  0.51886117  0.48477764  0.48921884  0.49304354
  0.50734913  0.49077084  0.48238647  0.51116222]


In [11]:
len(predictions)

1250

Here we have printe only the first ten probabilities. Note: these values correspond to the probability of the market going down, rather than up. If we print the model's encoding of the response values alongside the original nominal response, we see that Python has created a dummy variable with
a 1 for ${\tt Down}$.

In [12]:
print(np.column_stack((df.as_matrix(columns=["Direction"]).flatten(), result.model.endog)))

[['Up' 0.0]
 ['Up' 0.0]
 ['Down' 1.0]
 ..., 
 ['Up' 0.0]
 ['Down' 1.0]
 ['Down' 1.0]]


In order to make a prediction as to whether the market will go up or
down on a particular day, we must convert these predicted probabilities
into class labels, ${\tt Up}$ or ${\tt Down}$. The following two commands create a vector
of class predictions based on whether the predicted probability of a market
increase is greater than or less than 0.5.

In [13]:
predictions_nominal = [ "Up" if x < 0.5 else "Down" for x in predictions]

In [14]:
predictions_nominal

['Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Down',
 'Up',
 'Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Up',
 'Up',
 'Up',
 'Down',
 'Up',
 'Up',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Down',
 'Up',
 'Up',
 'Down',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Up',
 'Down',
 'Down',
 'Down',
 'Down',
 'Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Down',
 'Up',
 'Down',
 'Up',
 'Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Down',
 'Down',
 'Up',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Up',
 'Down',
 'Down',
 'Up',
 'Up',
 'Up',
 'Up',
 'Up',
 'Down',
 'Do

This transforms to ${\tt Up}$ all of the elements for which the predicted probability of a
market increase exceeds 0.5 (i.e. probability of a decrease is below 0.5). Given these predictions, the ${\tt confusion\_matrix()}$ function can be used to produce a confusion matrix in order to determine how many
observations were correctly or incorrectly classified.

In [15]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df["Direction"], predictions_nominal))

[[145 457]
 [141 507]]


The diagonal elements of the confusion matrix indicate correct predictions,
while the off-diagonals represent incorrect predictions. Hence our model
correctly predicted that the market would go up on 507 days and that
it would go down on 145 days, for a total of 507 + 145 = 652 correct
predictions. The ${\tt mean()}$ function can be used to compute the fraction of
days for which the prediction was correct. In this case, logistic regression
correctly predicted the movement of the market 52.2% of the time. this is confirmed by checking the output of the ${\tt classification\_report()}$ function.

In [16]:
print(classification_report(df["Direction"], predictions_nominal, digits=3))

             precision    recall  f1-score   support

       Down      0.507     0.241     0.327       602
         Up      0.526     0.782     0.629       648

avg / total      0.517     0.522     0.483      1250



At first glance, it appears that the logistic regression model is working
a little better than random guessing. But remember, this result is misleading
because we trained and tested the model on the same set of 1,250 observations.
In other words, 100− 52.2 = 47.8% is the **training error rate**. As we
have seen previously, the training error rate is often overly optimistic — it
tends to underestimate the _test_ error rate. 

In order to better assess the accuracy
of the logistic regression model in this setting, we can fit the model
using part of the data, and then examine how well it predicts the held out
data. This will yield a more realistic error rate, in the sense that in practice
we will be interested in our model’s performance not on the data that
we used to fit the model, but rather on days in the future for which the
market’s movements are unknown.

Like we did with KNN, we will first create a vector corresponding
to the observations from 2001 through 2004. We will then use this vector
to create a held out data set of observations from 2005.

In [17]:
x_train = df[:'2004'][:]
y_train = df[:'2004']['Direction']

x_test = df['2005':][:]
y_test = df['2005':]['Direction']

We now fit a logistic regression model using only the subset of the observations
that correspond to dates before 2005, using the subset argument.
We then obtain predicted probabilities of the stock market going up for
each of the days in our test set—that is, for the days in 2005.

In [28]:
x_train

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
2001-01-01,0.213,0.614,-0.623,1.032,0.959,1.34910,1.392,Up
2001-01-01,1.392,0.213,0.614,-0.623,1.032,1.44500,-0.403,Down
2001-01-01,-0.403,1.392,0.213,0.614,-0.623,1.40780,0.027,Up
2001-01-01,0.027,-0.403,1.392,0.213,0.614,1.16400,1.303,Up


In [18]:
model = smf.glm(formula=formula, data=x_train, family=sm.families.Binomial())
result = model.fit()

Notice that we have trained and tested our model on two completely separate
data sets: training was performed using only the dates before 2005,
and testing was performed using only the dates in 2005. Finally, we compute
the predictions for 2005 and compare them to the actual movements
of the market over that time period.

In [19]:
predictions = result.predict(x_test)
predictions_nominal = [ "Up" if x < 0.5 else "Down" for x in predictions]
print(classification_report(y_test, predictions_nominal, digits=3))

             precision    recall  f1-score   support

       Down      0.443     0.694     0.540       111
         Up      0.564     0.312     0.402       141

avg / total      0.511     0.480     0.463       252



The results are rather disappointing: the test error
rate (1 - ${\tt recall}$) is 52%, which is worse than random guessing! Of course this result
is not all that surprising, given that one would not generally expect to be
able to use previous days’ returns to predict future market performance.
(After all, if it were possible to do so, then the authors of this book [along with your professor] would probably
be out striking it rich rather than teaching statistics.)

We recall that the logistic regression model had very underwhelming pvalues
associated with all of the predictors, and that the smallest p-value,
though not very small, corresponded to ${\tt Lag1}$. Perhaps by removing the
variables that appear not to be helpful in predicting ${\tt Direction}$, we can
obtain a more effective model. After all, using predictors that have no
relationship with the response tends to cause a deterioration in the test
error rate (since such predictors cause an increase in variance without a
corresponding decrease in bias), and so removing such predictors may in
turn yield an improvement. 

In the space below, refit a logistic regression using just ${\tt Lag1}$ and ${\tt Lag2}$, which seemed to have the highest predictive power in the original logistic regression model.

In [32]:
x_train1 = x_train[['Lag1', 'Lag2','Direction']]
x_test1 = x_test[['Lag1', 'Lag2','Direction']]

formula = 'Direction ~ Lag1+Lag2'
model = smf.glm(formula=formula, data=x_train1, family=sm.families.Binomial())
# This will test your new model
result = model.fit()
predictions = result.predict(x_test1)
predictions_nominal = [ "Up" if x < 0.5 else "Down" for x in predictions]
print(classification_report(y_test, predictions_nominal, digits=3))

             precision    recall  f1-score   support

       Down      0.500     0.315     0.387       111
         Up      0.582     0.752     0.656       141

avg / total      0.546     0.560     0.538       252



Now the results appear to be more promising: 56% of the daily movements
have been correctly predicted. The confusion matrix suggests that on days
when logistic regression predicts that the market will decline, it is only
correct 50% of the time. However, on days when it predicts an increase in
the market, it has a 58% accuracy rate.

Finally, suppose that we want to predict the returns associated with **particular
values** of ${\tt Lag1}$ and ${\tt Lag2}$. In particular, we want to predict Direction on a
day when ${\tt Lag1}$ and ${\tt Lag2}$ equal 1.2 and 1.1, respectively, and on a day when
they equal 1.5 and −0.8. We can do this by passing a new data frame containing our test values to the ${\tt predict()}$ function.

In [33]:
print(result.predict(pd.DataFrame([[1.2,1.1],[1.5,-0.8]], columns = ["Lag1","Lag2"])))

0    0.520854
1    0.503906
dtype: float64
